<img style="float: right;" width="120" src="../Images/supplier-logo.png">
<img style="float: left; margin-top: 0" width="80" src="../Images/client-logo.png">
<br><br><br>


# Synopsis

This notebook will explain the following topics and concepts:

**Ranking Data** 
- Ascending and Descending

**Concatenation**
- Rows and Columns

**Merging Data**
- left
- right
- inner
- outer

**Joining Data**
- left
- right
- inner
- outer

**Grouping Data**
- by time
- by columns

# Importing libraries & Load Data

We'll use the same csv files as we used in chapter 3.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


# format for floats
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
df_GOOGL = pd.read_excel('../Data/market_data.xls', sheet_name='GOOGL', index_col='Date', parse_dates=True)
df_IBM = pd.read_excel('../Data/market_data.xls', sheet_name='IBM', index_col='Date', parse_dates=True)
df_MSFT = pd.read_excel('../Data/market_data.xls', sheet_name='MSFT', index_col='Date', parse_dates=True)

# Ranking Data

- We can compute a numerical rank from one through to the number of valid data points.
- This can be applied to Series and DataFrames.
- Use the **rank()** method.
- Often used in conjunction with sort.
- In the case of a tie, the default behaviour is to assign the mean to each tied group.

Let's try a simple example to clarify the concept of ranking
  - We'll add a column called rank to the IBM DataFrame, based on the AdjVolume.<br>
  - This will create a new column called 'rank'.
  - The 'rank' column will contain a number from 0 to n for each row, indicating the rank of that row's AdjVolume.

In [ ]:
df_IBM['rank'] = df_IBM['Volume'].rank()
display(df_IBM.head())

Now, if we sort the table according to the 'rank' column then it should also be sorted according to 'Volume'.

In [ ]:
display(df_IBM.sort_values(by="rank").head())

Let's try a more realistic example:

Rank the difference in AdjVol between Google and IBM, for all days in 2017 where AdjVol for Google > AdjVol for IBM

There are a few ways to achieve this, the simplest is to create a new DataFrame to store all the adjusted Volumes for IBM and Google for the year 2017 and then use this to filter and rank.

## Note that 
for most days in 2017, the Volume for IBM > Volume for Google

In [ ]:
df_IBM['Volume']['2017'].plot()
df_GOOGL['Volume']['2017'].plot()

Rank the days where Volume for Google > Volumne for IBM< for the year 2017

In [ ]:
# A few ways to achieve this
# Easiest to understand is create a new DataFrame for holding Volumes
df_Vol = pd.DataFrame()

# Add the AdjVolume for 2017 of IBM and Google as separate columns in our new DataFrame
df_Vol['IBM'] = df_IBM['2017']['Volume']
df_Vol['Google'] = df_GOOGL['2017']['Volume']

# Create a new column containing the difference
df_Vol['Diff'] = df_Vol['Google'] - df_Vol['IBM']

# Now use a filter (same as in lesson 3) to find where Google is greater than IBM
goog_gt_ibm = df_Vol['Google'] > df_Vol['IBM']

# Get the rows in Google's AdjVolume that match the filter and rank them
# We haven't sorted them so they're still displayed in order of the index (Date)
df_Vol[['Diff']][goog_gt_ibm].rank()

In [ ]:
# store the results of the filter in a temporary DataFrame
# Note the use of copy - create a copy of the data rather than a reference to it
df_tmp = df_Vol[goog_gt_ibm].copy()

# Calculate the differences and store these values as a new column
df_tmp['Diff)'] = df_tmp['Google'] - df_tmp['IBM']

# Add a new ranking column
df_tmp['Rank'] = df_tmp['Diff)'].rank()

# Sort
df_tmp.sort_values(by='Rank')

We could also rank in the other direction (highest number gets a rank of 1)

In [ ]:
# Rank top to Bottom and Sort
df_tmp['Rank'] = df_tmp['Diff'].rank(ascending=False)
df_tmp.sort_values(by='Rank')

# Concatenation

- Glues together DataFrames, without much intelligence.
- Dimensions should match along the axis you are concatenating on. 
- Use **pd.concat** and pass in a list of DataFrames to concatenate together

## Create a few simple DataFrames 

In [ ]:
date_range = pd.date_range(start='2017', freq='BQ', periods=4)
cols  = ['Open', 'Close']

df1 = df_IBM.reindex(date_range)[cols]
df2 = df_GOOGL.reindex(date_range)[cols]
df3 = df_MSFT.reindex(date_range)[cols]

## Concatenate

The default is to concatenate the rows

In [ ]:
df_all = pd.concat([df1, df2, df3])
df_all

## Concatenate the columns
Pass the `axis = 1` parameter to pd.concat

In [ ]:
df_all = pd.concat([df1, df2, df3], axis=1)
df_all


# Merging

Pandas has two important functions for joining DataFrames together which intelligently try to align values from selected columns of each DataFrame. These functions are called **merge** and **join**. These functions use a similar logic to joins in SQL.

First we will look at merge.

**There are 4 Different types of merge**
- **Inner Merge** – The default Pandas behaviour, only keep rows where the merge “on” value exists in both the left and right DataFrames.
- **Left Merge** – (aka left merge or left join) Keep every row in the left DataFrame. Where there are missing values of the “on” variable in the right DataFrame, add empty / NaN values in the result.
- **Right Merge** – (aka right merge or right join) Keep every row in the right DataFrame. Where there are missing values of the “on” variable in the left column, add empty / NaN values in the result.
- **Outer Merge** – A full outer join returns all the rows from the left DataFrame, all the rows from the right DataFrame, and matches up rows where possible, with NaNs elsewhere.

## Create some sample DataFrames

Just a few days worth of Data from Google and IBM

Note the difference in date ranges

In [ ]:
cols = ['High', 'Low']

df1 = df_IBM[cols]['2017-Jan-01':'2017-Jan-06'].sort_index()
df2 = df_GOOGL[cols]['2017-Jan-05':'2017-Jan-10'].sort_index()

# show both dataframes
print("== IBM ==")
display(df1)
print("== GOOGLE ==")
display(df2)

## Inner Merge

Only keep values for Dates found in both left (df1) and right (df2)

In [ ]:
# This version of merge works in latest Pandas
pd.merge(df1, df2, how='inner', on='Date')

# This version of merge works in older Pandas
pd.merge(df1, df2, how='inner', left_index=True, right_index=True)


## Left Merge

- Keep everything in the left DataFrame.
- Where nothing exists in the right DataFrame, fill with NaN ("Not a Number" - these are empty values).
- Use the suffixes parameter to override the x_ and y_ defaults

In [ ]:
srcs = ['_IBM','_GOOGL']

pd.merge(df1, df2, how='left', on='Date', suffixes=srcs)

pd.merge(df1, df2, how='left', left_index=True, right_index=True,  suffixes=srcs)

## Right Merge

- Keep everything in the right DataFrame
- Where nothing exists in the left DataFrame, fill with NaN

In [ ]:
pd.merge(df1, df2, how='right', on='Date', suffixes=srcs)

pd.merge(df1, df2, how='right', left_index=True, right_index=True, suffixes=srcs)

## Outer Merge

Keep everything in both left and right DataFrames, fill with NaN where no data present

In [ ]:
pd.merge(df1, df2, how='outer', on='Date', suffixes=srcs)

pd.merge(df1, df2, how='outer', on='Date', left_index=True, right_index=True, suffixes=srcs)

# Joining

- The second pandas function for intelligently combining DataFrames is called **join**.
- Join is **very** similar to merge.
- As with merge, the **how** parameter takes inner, outer, left or right.
- As with merge, the **on** parameter is the name of a column to join on.
- However there is one major difference:
  - When using join the "on" **must** be the index in at least one of the DataFrames.
  - Merge will allow the "on" to be a regular column in **both** DataFrames.

The syntax for calling the two functions is also slightly different:
- **join**  : df1.join(df2, how="inner", on="Date")
- **merge** : pd.merge(df1, df2, how="inner", on="Date")



In [ ]:
cols1 = ['High', 'Low']
cols2 = ['Open', 'Close']
df1 = df_IBM[cols1]['2017-Jan-01':'2017-Jan-07'].sort_index()
df2 = df_IBM[cols2]['2017-Jan-04':'2017-Jan-11'].sort_index()

# show both dataframes
print("== High & Low ==")
display(df1)
print("== Open & Close ==")
display(df2)



In [ ]:
# If we don't specify an "on" then we will join "on" the index of both DataFrames
df1.join(df2)


In [ ]:
df1.join(df2, how='right')


In [ ]:
df1.join(df2, how='inner')


In [ ]:
df1.join(df2, how='outer')


# Grouping Data

Pandas provides functions that allow us to group rows of data together and call aggregate functions on them as a unit e.g. mean, max, min, std, etc.

To create a group we call the **groupby** method on a DataFrame.

e.g. Create groups where the 'Industry' column is the same:
    - df1.groupby('Industry')

## Group by Columns

Use the **by** parameter and supply a column name

In [ ]:
df = pd.read_excel(io='../Data/sample_data.xls', sheet_name='Groups', index_col='Date', parse_dates=True)

In [ ]:
# Grouping is a convenient way to get the mean for each sector of each column
df.groupby(by='Sector').mean()

# Grouping is a convenient way to get the mean for each sector of each column
df.groupby(by='Rep').mean()

# Grouping is a convenient way to get the mean for each sector of each column
df.groupby(by='Portfolio').mean()

# Or we could create a variable to store the name of the function
func = 'std'
df.groupby(by='Sector').agg(func)

# Or we can create a list of functions to aggregate with
funcs = ['median', 'std']
df.groupby(by='Rep').agg(funcs)


## Group by Time Period
Often we want to group according to a frequency e.g. a group of all values in a single business quarter.

We can then call mean for all of the groups, e.g. get the mean Volume per business quarter.

A convenient way to group by a frequency is to use the Grouper object (in the pandas namespace).

- pd.Grouper
- Most commonly used to pass in a frequency
- Group by frequencies: B (business day), BQ (business quarter), M (month), Y (year), etc.
- It's also possible to group by specific time frequencies e.g. 15d, 1h30min etc.
  - See the list of frequency aliases: http://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases

In [ ]:
df.groupby(pd.Grouper(freq='BQ')).mean()

# Or we could create a variable to store the grouper
by_BMonth = pd.Grouper(freq='BM')
df.groupby(by=by_BMonth).mean()

# Or use teh agg and a list of functions
funcs = ['median', 'std']
df.groupby(by=by_BMonth).agg(funcs)


# Finally we can supply a list of groupers
by_BMonth = pd.Grouper(freq='BM')
by_Rep = pd.Grouper(key='Rep')
by_Sector = pd.Grouper(key='Sector')

# And perform a variety of slice and dice operations
groups1 = [by_Rep, by_BMonth, by_Sector]
df.groupby(by=groups1).agg(funcs)

groups2 = [by_Sector, by_Rep, by_BMonth]
df.groupby(by=groups2).agg(funcs)

groups3 = [by_BMonth, by_Sector, by_Rep]
df.groupby(by=groups3).agg(funcs)

In [ ]:
df_GOOGL['Adj. Open'].groupby(pd.Grouper(freq='Q')).mean()
df_GOOGL['Adj. Open'].groupby(pd.Grouper(freq='Q')).mean().plot()

df_GOOGL['2017']['Adj. Open'].groupby(pd.Grouper(freq='d')).mean()
df_GOOGL['2017']['Adj. Open'].rolling(30).mean()
df_GOOGL['2017']['Adj. Open'].rolling(30, min_periods=3).mean()

df_GOOGL['2017']['Adj. Open'].groupby(pd.Grouper(freq='15d')).mean()
df_GOOGL['Adj. Open'].groupby(pd.Grouper(freq='BQ')).mean()

df_GOOGL['Adj. Open'].groupby(pd.Grouper(freq='BQ')).mean()

In [ ]:
rng = pd.date_range(start='2019', freq='BM', periods=12)
df = pd.read_excel(io='../Data/sample_data.xls', sheet_name='Groups')
del df['Price']


In [ ]:
rng

In [ ]:
df_new = pd.DataFrame()

for i in rng:
    tmp = df.copy()
    tmp.insert(loc=0, column='Date', value=i)
    his = np.random.randint(98, 103, tmp['High'].shape[0]) / 100
    tmp['High'] = tmp['High'] * his
    tmp['Low'] = tmp['Low'] * 0.98
    df_new = pd.concat([df_new, tmp])
    
pd.concat([df_new.head(), df_new.tail()])

In [ ]:
df_new.index = df_new['Date']
del df_new['Date']

In [ ]:
np.random.randint(98, 103, 5) / 100

In [ ]:
pd.Grouper?

In [ ]:
#w = pd.ExcelWriter(path='../Data/sample_data.xls')

#df_new.to_excel(excel_writer=w, sheet_name='NewGroups')

#w.save()